In [ ]:
!pip install happytransformer

In [ ]:
from happytransformer import HappyTextToText, TTSettings,TTTrainArgs

from datasets import load_dataset
from happytransformer import HappyTextToText
import pandas as pd


## Download the model "t5-base"

In [ ]:

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")





/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Load Dataset

In [ ]:
train_path = "./drive/MyDrive/final-year-project/train.csv"
eval_path = "./drive/MyDrive/final-year-project/eval.csv"

In [ ]:
def load_data(path):
  return pd.read_csv(path)


train_data = load_data(train_path)
eval_data = load_data(eval_path)

In [ ]:
train_data.shape

(755, 2)

In [ ]:
eval_data.shape

(748, 2)

In [ ]:
train_data

,sentence,corrections
0,So I think we can not live if old people could...,['So I think we would not be alive if our ance...
1,For not use car .,"['Not for use with a car . ', 'Do not use in t..."
2,Here was no promise of morning except that we ...,"['Here was no promise of morning , except that..."
3,Thus even today sex is considered as the least...,"['Thus , even today , sex is considered as the..."
4,image you salf you are wark in factory just to...,"[""Imagine yourself you are working in factory ..."
...,...,...
750,The government also should try to reduce the s...,['The government should also try to reduce the...
751,Alot of memories with enogh time to remember w...,"['A lot of memories , with enough time to reme..."
752,Sceene of violence can affect on them .,['A scene of violence can have an effect on th...
753,While the communities in general have reckoned...,['The communities in general have reckoned tha...


In [ ]:
train_data.isnull().sum()

sentence       1
corrections    0
dtype: int64

In [ ]:
eval_data.isnull().sum()

sentence       1
corrections    0
dtype: int64

In [ ]:
train_data[train_data["sentence"].isna()]

,sentence,corrections
754,NaN,"['', '', '', '']"


In [ ]:
eval_data[eval_data["sentence"].isna()]

,sentence,corrections
747,NaN,"['', '', '', '']"


In [ ]:
train_data[train_data["sentence"].apply(lambda x:  len(str(x)) < 5  )]
eval_data[eval_data["sentence"].apply(lambda x:  len(str(x)) < 5  )]

,sentence,corrections
747,NaN,"['', '', '', '']"


## clean and Preprocess Data

In [ ]:
import ast

In [ ]:
train_data["corrections"].dtype

dtype('O')

In [ ]:
train_data["corrections"].apply(ast.literal_eval)

0      [So I think we would not be alive if our ances...
1      [Not for use with a car . , Do not use in the ...
2      [Here was no promise of morning , except that ...
3      [Thus , even today , sex is considered as the ...
4      [Imagine yourself you are working in factory j...
                             ...                        
750    [The government should also try to reduce the ...
751    [A lot of memories , with enough time to remem...
752    [A scene of violence can have an effect on the...
753    [The communities in general have reckoned that...
754                                             [, , , ]
Name: corrections, Length: 755, dtype: object

In [ ]:
# remove null_values
def remove_unclean_data(df:pd.DataFrame)->pd.DataFrame:
  return df[ ~ df["sentence"].isnull()]


#convert data from list to string
def convert_list_to_string(df:pd.DataFrame)->pd.DataFrame:
  df["corrections"] = train_data["corrections"].apply(ast.literal_eval)
  return df.explode("corrections", ignore_index=True)



In [ ]:
train_data

,sentence,corrections
0,So I think we can not live if old people could...,['So I think we would not be alive if our ance...
1,For not use car .,"['Not for use with a car . ', 'Do not use in t..."
2,Here was no promise of morning except that we ...,"['Here was no promise of morning , except that..."
3,Thus even today sex is considered as the least...,"['Thus , even today , sex is considered as the..."
4,image you salf you are wark in factory just to...,"[""Imagine yourself you are working in factory ..."
...,...,...
750,The government also should try to reduce the s...,['The government should also try to reduce the...
751,Alot of memories with enogh time to remember w...,"['A lot of memories , with enough time to reme..."
752,Sceene of violence can affect on them .,['A scene of violence can have an effect on th...
753,While the communities in general have reckoned...,['The communities in general have reckoned tha...


In [ ]:
train_df = convert_list_to_string(remove_unclean_data(train_data))
eval_df = convert_list_to_string(remove_unclean_data(eval_data))




<ipython-input-17-52213e9c70f2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["corrections"] = train_data["corrections"].apply(ast.literal_eval)
<ipython-input-17-52213e9c70f2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["corrections"] = train_data["corrections"].apply(ast.literal_eval)


In [ ]:
train_df.head()

,sentence,corrections
0,So I think we can not live if old people could...,So I think we would not be alive if our ancest...
1,So I think we can not live if old people could...,So I think we could not live if older people d...
2,So I think we can not live if old people could...,So I think we can not live if old people could...
3,So I think we can not live if old people could...,So I think we can not live if old people can n...
4,For not use car .,Not for use with a car .


In [ ]:
train_df.isnull().sum()

sentence       0
corrections    0
dtype: int64

In [ ]:
eval_df.isnull().sum()

sentence       0
corrections    0
dtype: int64

In [ ]:
#prefix "grammar:" to the sentence column

def prefix_grammar(df:pd.DataFrame)-> pd.DataFrame:
  df["sentence"] = df["sentence"].apply(lambda x: f"grammar: {x}")
  return df

In [ ]:
train_df = prefix_grammar(train_df)
eval_df = prefix_grammar(eval_df)

train_df.head()

,sentence,corrections
0,grammar: So I think we can not live if old peo...,So I think we would not be alive if our ancest...
1,grammar: So I think we can not live if old peo...,So I think we could not live if older people d...
2,grammar: So I think we can not live if old peo...,So I think we can not live if old people could...
3,grammar: So I think we can not live if old peo...,So I think we can not live if old people can n...
4,grammar: For not use car .,Not for use with a car .


###  Rename the columns to input target(format for the model)

In [ ]:
def rename_the_columns_and_save_it(df:pd.DataFrame,name:str):
   df.rename(columns={"sentence":"input","corrections":"target"},inplace=True)
   df.to_csv(f"./{name}.csv",index=False)

In [ ]:
rename_the_columns_and_save_it(train_df,"train")
rename_the_columns_and_save_it(eval_df,"eval")

In [ ]:
before_training = happy_tt.eval("eval.csv")

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

In [ ]:
print("before training:", before_training.loss)

before training: 4.592572212219238


In [ ]:
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv",args=args)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1,0.824500,0.558628
34,0.441700,0.383367
68,0.427300,0.363028
102,0.451700,0.355197
136,0.409400,0.349850
170,0.419300,0.347365
204,0.391600,0.349663
238,0.421100,0.343870
272,0.403500,0.343161
306,0.471100,0.342090


In [ ]:
after_training = happy_tt.eval("eval.csv")
print("after training:", after_training.loss)

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

after training: 4.330343723297119


In [ ]:
print(f"loss before_training: before_training.loss")
print(f"loss after training:  after_training.loss")

loss before_training: before_training.loss
loss after training:  after_training.loss


In [ ]:

example_1 = "grammar: she say she hate short guys"
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)
result_1 = happy_tt.generate_text(example_1,args=beam_settings)
print(result_1.text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


She says she hates short guys.


## Save Model

In [ ]:


happy_tt.save("./drive/MyDrive/final-year-model")